# Brain Tumor Classification

## Initialization

### Imports

In [ ]:
import os 
import numpy as np
import torch

### Seeds and Init

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    print("Using GPU")
else:
    print("Using CPU")

torch.backends.cudnn.benchmark = True
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)

## Data Exploration

In [ ]:
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

Task 1: We need to explore the dataset. Start off by printing the number of images in each class. We need to know how many images there are in each class for both the training and testing sets. Next, we need to create a function that allows us to visualize the images in the dataset randomly (using the os library). For this visualization, we need to use matplotlib to visualize and the title of the image should be the class of the image. 

## Data Preprocessing

Task 2: We need to work on preprocessing the dataset. The best way to do this would be to first load the dataset by making it into a dataset object and then applying transformations using PyTorch.

In [ ]:
class MRIData(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data 

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, id):
        return self.data[id]